In [1]:
from sklearn.model_selection import GridSearchCV 
import pickle
from sklearn.utils import shuffle
from sklearn.svm import SVC
import numpy as np
from sklearn import svm
from sklearn import metrics
import pickle
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [2]:
## data load
data = pd.read_csv('feats.csv', index_col=0)
data.head(6)


,index,total_max,r,g,b,r_d,g_d,b_d,peri,x,...,circ,lbp_energy,lbp_entropy,contrast,dissimilarity,homogeneity,energy,correlation,gabor_energy,gabor_entropy
0,0,59,244.472007,244.637854,244.475990,3.290990,3.232540,3.261062,6352.242091,102,...,22.492034,0.151042,2.766991,83.333815,4.832388,0.313027,0.059691,0.995079,0.460289,1.481977
0,0,54,248.732345,248.916303,248.724208,3.048503,3.054855,3.040140,5194.467491,49,...,21.271667,0.149276,2.775581,91.714158,5.120396,0.310293,0.063597,0.994339,0.478135,1.470162
0,0,64,244.694745,244.733385,244.509274,3.073011,3.097050,3.124836,4934.092827,123,...,20.731387,0.150752,2.768430,80.212163,4.672703,0.329030,0.067146,0.994962,0.438472,1.534476
0,0,74,248.465111,248.711508,248.689545,3.332536,3.280357,3.401647,5936.859409,157,...,21.411939,0.150419,2.770009,75.524779,4.474735,0.345216,0.085150,0.995488,0.533779,1.319005
0,0,54,248.368230,248.553253,248.428208,2.948238,2.930055,2.970568,3662.456181,90,...,19.218754,0.148587,2.779272,79.923764,4.567175,0.345007,0.075264,0.994896,0.515664,1.362576
0,0,53,251.744972,251.880616,251.761232,2.844268,2.835204,2.855736,2679.103802,98,...,18.536297,0.151795,2.763754,77.371396,3.964674,0.368731,0.096603,0.995866,0.616675,1.069170


In [3]:
y = data.iloc[:,:1]
y = np.array(y)

In [4]:
train_feats = data.iloc[:,1:]
train_feats
x = np.array(train_feats)

In [5]:
# Normalizing the train features to the range [0,1]
trMaxs = np.amax( x , axis=0)
trMins = np.amin( x , axis=0) 
trMaxs_rep = np.tile(trMaxs,(len(x),1)) 
trMins_rep = np.tile(trMins,(len(x),1)) 
x = np.divide(x-trMins_rep,trMaxs_rep)   

In [6]:
train_feat, val_feat, train_label, val_label = train_test_split(x,y, test_size=0.3, random_state = 142)

In [7]:
# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4, 1e-5],'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000,1100,1200,1300,1500,2000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for {}".format(score))
 
    clf_r = GridSearchCV(SVC(C=1), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf_r.fit(train_feat, train_label.ravel())   

# Tuning hyper-parameters for precision
# Tuning hyper-parameters for recall


In [8]:
print("Best parameters set found on sets: {} \n".format(clf_r.best_params_))
print("scores on sets:")
means = clf_r.cv_results_['mean_test_score']
stds = clf_r.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf_r.cv_results_['params']):
    print("mean: {}  std: (+/-{}) params: {}".format(mean, std * 2, params))

Best parameters set found on sets: {'C': 2000, 'gamma': 0.01, 'kernel': 'rbf'} 

scores on sets:
mean: 0.0625  std: (+/-0.0) params: {'C': 0.001, 'gamma': 0.01, 'kernel': 'rbf'}
mean: 0.0625  std: (+/-0.0) params: {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'}
mean: 0.0625  std: (+/-0.0) params: {'C': 0.001, 'gamma': 0.0001, 'kernel': 'rbf'}
mean: 0.0625  std: (+/-0.0) params: {'C': 0.001, 'gamma': 1e-05, 'kernel': 'rbf'}
mean: 0.0625  std: (+/-0.0) params: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
mean: 0.0625  std: (+/-0.0) params: {'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}
mean: 0.0625  std: (+/-0.0) params: {'C': 0.1, 'gamma': 0.0001, 'kernel': 'rbf'}
mean: 0.0625  std: (+/-0.0) params: {'C': 0.1, 'gamma': 1e-05, 'kernel': 'rbf'}
mean: 0.0625  std: (+/-0.0) params: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
mean: 0.0625  std: (+/-0.0) params: {'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}
mean: 0.0625  std: (+/-0.0) params: {'C': 0.1, 'gamma': 0.0001, 'kernel': 'rbf'}
mean: 0.0625  s

In [9]:
y_true, y_pred = val_label, clf_r.predict(val_feat)
metrics.accuracy_score(y_true, y_pred)

0.8388888888888889

In [10]:
print("classification scores:")
print(metrics.classification_report(y_true, y_pred))

classification scores:
             precision    recall  f1-score   support

          0       0.27      0.38      0.31        21
          1       1.00      1.00      1.00        22
          2       0.80      0.59      0.68        34
          3       0.95      0.95      0.95        22
          4       0.96      1.00      0.98        25
          5       1.00      0.88      0.93        24
          6       0.23      0.27      0.25        22
          7       1.00      1.00      1.00        21
          8       0.95      0.86      0.90        21
          9       0.82      0.72      0.77        25
         10       1.00      1.00      1.00        23
         11       1.00      1.00      1.00        13
         12       1.00      1.00      1.00        23
         13       0.91      1.00      0.95        20
         14       0.96      1.00      0.98        22
         15       0.95      0.95      0.95        22

avg / total       0.86      0.84      0.85       360



In [11]:
with open("train_model_rbf.pckl", "wb") as f:
    pickle.dump(clf_r, f)

In [12]:
tuned_parameters = [{'kernel': ['sigmoid'], 'gamma': [1e-2, 1e-3, 1e-4, 1e-5],'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000,1100,1200,1300,2000]}]



scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for {}".format(score))

    clf_s = GridSearchCV(SVC(C=1), tuned_parameters, cv=5, scoring='%s_macro' % score)
    clf_s.fit(train_feat, train_label.ravel())

# Tuning hyper-parameters for precision
# Tuning hyper-parameters for recall


In [13]:
print("Best parameters set found on sets: {} \n".format(clf_s.best_params_))
print("scores on sets:")
means = clf_s.cv_results_['mean_test_score']
stds = clf_s.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf_s.cv_results_['params']):
    print("mean: {}  std; (+/-{}) params: {}".format(mean, std * 2, params))

Best parameters set found on sets: {'C': 2000, 'gamma': 0.01, 'kernel': 'sigmoid'} 

scores on sets:
mean: 0.0625  std; (+/-0.0) params: {'C': 0.001, 'gamma': 0.01, 'kernel': 'sigmoid'}
mean: 0.0625  std; (+/-0.0) params: {'C': 0.001, 'gamma': 0.001, 'kernel': 'sigmoid'}
mean: 0.0625  std; (+/-0.0) params: {'C': 0.001, 'gamma': 0.0001, 'kernel': 'sigmoid'}
mean: 0.0625  std; (+/-0.0) params: {'C': 0.001, 'gamma': 1e-05, 'kernel': 'sigmoid'}
mean: 0.0625  std; (+/-0.0) params: {'C': 0.1, 'gamma': 0.01, 'kernel': 'sigmoid'}
mean: 0.0625  std; (+/-0.0) params: {'C': 0.1, 'gamma': 0.001, 'kernel': 'sigmoid'}
mean: 0.0625  std; (+/-0.0) params: {'C': 0.1, 'gamma': 0.0001, 'kernel': 'sigmoid'}
mean: 0.0625  std; (+/-0.0) params: {'C': 0.1, 'gamma': 1e-05, 'kernel': 'sigmoid'}
mean: 0.0625  std; (+/-0.0) params: {'C': 0.1, 'gamma': 0.01, 'kernel': 'sigmoid'}
mean: 0.0625  std; (+/-0.0) params: {'C': 0.1, 'gamma': 0.001, 'kernel': 'sigmoid'}
mean: 0.0625  std; (+/-0.0) params: {'C': 0.1, 'gamm

In [14]:
y_true, y_pred = val_label, clf_s.predict(val_feat)
metrics.accuracy_score(y_true, y_pred)

0.8361111111111111

In [15]:
print("classification scores:")
print(metrics.classification_report(y_true, y_pred))

classification scores:
             precision    recall  f1-score   support

          0       0.31      0.52      0.39        21
          1       1.00      1.00      1.00        22
          2       0.79      0.56      0.66        34
          3       0.95      0.91      0.93        22
          4       0.96      1.00      0.98        25
          5       1.00      0.88      0.93        24
          6       0.19      0.18      0.19        22
          7       1.00      1.00      1.00        21
          8       0.95      0.86      0.90        21
          9       0.82      0.72      0.77        25
         10       1.00      1.00      1.00        23
         11       0.93      1.00      0.96        13
         12       1.00      1.00      1.00        23
         13       0.91      1.00      0.95        20
         14       1.00      1.00      1.00        22
         15       0.91      0.95      0.93        22

avg / total       0.86      0.84      0.84       360



In [16]:
with open("train_model_sigmoid.pckl", "wb") as f:
    pickle.dump(clf_s, f)

In [17]:
tuned_parameters = [{'kernel': ['linear'], 'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000 ,1200,1300,1500,2000,2100,2200,2300]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for {}".format( score))
    print()

    clf_l = GridSearchCV(SVC(C=1), tuned_parameters, cv=5, scoring='%s_macro' % score)
    clf_l.fit(train_feat, train_label.ravel())

# Tuning hyper-parameters for precision

# Tuning hyper-parameters for recall



In [18]:
print("Best parameters set found on sets: {} \n".format(clf_l.best_params_))
print("scores on sets:")
    
means = clf_l.cv_results_['mean_test_score']
stds = clf_l.cv_results_['std_test_score']
    
for mean, std, params in zip(means, stds, clf_l.cv_results_['params']):
    print("mean: {}  std: (+/-{}) params {}".format(mean, std * 2, params))

Best parameters set found on sets: {'C': 1200, 'kernel': 'linear'} 

scores on sets:
mean: 0.0625  std: (+/-0.0) params {'C': 0.001, 'kernel': 'linear'}
mean: 0.29616612554112554  std: (+/-0.04432963581378538) params {'C': 0.1, 'kernel': 'linear'}
mean: 0.29616612554112554  std: (+/-0.04432963581378538) params {'C': 0.1, 'kernel': 'linear'}
mean: 0.7923698697136197  std: (+/-0.056723475826388446) params {'C': 10, 'kernel': 'linear'}
mean: 0.8138062198218446  std: (+/-0.09155265157470165) params {'C': 25, 'kernel': 'linear'}
mean: 0.8376711309523809  std: (+/-0.05863151765416371) params {'C': 50, 'kernel': 'linear'}
mean: 0.8396253486097236  std: (+/-0.050642890773262075) params {'C': 100, 'kernel': 'linear'}
mean: 0.8510712811494061  std: (+/-0.03395653776184592) params {'C': 1000, 'kernel': 'linear'}
mean: 0.8522144088550339  std: (+/-0.036579638491267205) params {'C': 1200, 'kernel': 'linear'}
mean: 0.8519961548867799  std: (+/-0.0435050126289358) params {'C': 1300, 'kernel': 'linear

In [19]:
y_true, y_pred = val_label, clf_l.predict(val_feat)
metrics.accuracy_score(y_true, y_pred)

0.8527777777777777

In [20]:
print("classification scores:")
print(metrics.classification_report(y_true, y_pred))

classification scores:
             precision    recall  f1-score   support

          0       0.20      0.19      0.20        21
          1       1.00      1.00      1.00        22
          2       0.93      0.82      0.87        34
          3       0.95      0.95      0.95        22
          4       0.96      1.00      0.98        25
          5       0.92      0.92      0.92        24
          6       0.22      0.27      0.24        22
          7       1.00      1.00      1.00        21
          8       0.90      0.90      0.90        21
          9       1.00      0.84      0.91        25
         10       1.00      1.00      1.00        23
         11       0.75      0.92      0.83        13
         12       1.00      1.00      1.00        23
         13       0.90      0.90      0.90        20
         14       1.00      0.95      0.98        22
         15       0.91      0.95      0.93        22

avg / total       0.86      0.85      0.86       360



In [21]:
with open("train_model_linear.pckl", "wb") as f:
    pickle.dump(clf_l, f)